# <font color = '#00CCFF'> High Value Customer Identification </font>

Project Objective: Find significant customers for the business who make high purchases of their favourite products. The organization wants to roll out a loyalty program to the high-value customers after identification of segments. Use the clustering methodology to segment customers into groups:

Dataset Description: This is a transnational dataset that contains all the transactions occurring between Nov-2016 to Dec-2017 for a UK-based online retail store.

Attribute Description:
- InvoiceNo: Invoice number (A 6-digit integral number uniquely assigned to each transaction)
- StockCode: Product (item) code
- Description: Product (item) name
- Quantity: The quantities of each product (item) per transaction
- InvoiceDate: The day when each transaction was generated
- UnitPrice: Unit price (Product price per unit)
- CustomerID: Customer number (Unique ID assigned to each customer)
- Country: Country name (The name of the country where each customer resides)

# 0. Imports

## 0.1. Libraries

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import scikitplot as skplt
import plotly.express as px
#%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

import umap.umap_ as umap

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

from ydata_profiling import ProfileReport

## 0.2. Functions

In [ ]:
# Save intermediate dataset
def save_interim(dataframe, file_name):
   dataframe.to_csv('../data/interim/'+file_name+'.zip', index=False, compression= 'zip')

# Load intermediate dataset
def load_interim(file_name):
   return pd.read_csv('../data/interim/'+file_name, compression='zip')

## 0.3. Load dataset

In [ ]:
path = '/home/ezequiel/Documentos/Comunidade_DS/insiders_clustering/data/raw/Ecommerce.zip'
df_raw = pd.read_csv(path, encoding='unicode_escape', compression='zip')
df_raw = df_raw.drop(columns=['Unnamed: 8'], axis=1)

In [ ]:
df_raw.head()

# 1. Descrição dos dados

In [ ]:
df1 = df_raw.copy()

## 1.1. Rename columns

In [ ]:
df1.columns

In [ ]:
cols = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date', 'unit_price', 'customer_id', 'country']
df1.columns = cols

## 1.2. Data dimensions

In [ ]:
print(f'Number of rows: {df1.shape[0]}')
print(f'Number of columns: {df1.shape[1]}')

## 1.3. Data types

InvoiceNo should be a number, but some records have letters

In [ ]:
df1.dtypes

## 1.4. Check NA

In [ ]:
df1.isna().sum()

## 1.5. Replace NA

In [ ]:
# Procurar se há algum customer_id "perdido" a partir do invoice_no
df_missing = df1[df1['customer_id'].isna()]
df_not_missing = df1[~df1['customer_id'].isna()]

In [ ]:
missing_invoice = df_missing['invoice_no'].drop_duplicates().tolist()

In [ ]:
# Os invoice_no dos customer_id faltantes não correspondem a nenhum invoice_no que possui um customer_id
df_not_missing.loc[df_not_missing['invoice_no'].isin(missing_invoice), :]

In [ ]:
df_not_missing['customer_id'].max()

In [ ]:
# Alternativa: atribuir um customer_id não utilizado para cada invoice_no único que não possui customer_id
aux = df_missing[['invoice_no']].drop_duplicates()
aux['customer_id'] = np.arange(19000, 19000+len(aux), 1)

df1 = pd.merge(df1, aux, on='invoice_no', how='left')
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])
df1.drop(columns=['customer_id_x', 'customer_id_y'], inplace=True)

In [ ]:
df1.isna().sum()

In [ ]:
df1 = df1.dropna(subset=['description', 'customer_id'])

## 1.6. Change dtyper

In [ ]:
df1.dtypes

In [ ]:
# invoice_date -> to datetime
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

# customer_id -> to int
df1['customer_id'] = df1['customer_id'].astype(int)

In [ ]:
df1.dtypes

In [ ]:
save_interim(dataframe=df1, file_name='df1.csv')

## 1.7. Descriptive statistics

In [ ]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(include='object')

### 1.7.1. Numerical attributes

In [ ]:
df_describe = num_attributes.describe().T
df_describe['range'] = df_describe['max'] - df_describe['min']
df_describe['median'] = num_attributes.median()
df_describe['skewness'] = num_attributes.skew()
df_describe['kurtosis'] = num_attributes.kurtosis()
df_describe.reset_index()

<font color='red'>**OBSERVAÇÕES:**</font>
- quantity com mínimo negativo (devolução?)
- quantity com valor max e min iguais em módulo
- unit_price com min = zero (promoção?)

### 1.7.2. Categorical attributes

#### Invoice_no

In [ ]:
# verifica se invoice_no contém, ao menos, uma letra e agrupa por invoice_no para o dataframe inteiro
aux = df1[df1['invoice_no'].str.contains('[a-zA-Z]')].groupby('invoice_no').count().reset_index()
print(f'Qtd de invoices únicos com letras: {len(aux)}')
print(f'Qtd total de invoices com letras {aux["stock_code"].sum()}')

In [ ]:
aux = len(df1.loc[df1['invoice_no'].str.contains('[a-zA-Z]'), ['quantity']])
print(f'Qtd de invoices com letras onde a quantidade é negativa: {aux}')

#### Stock_code

In [ ]:
# verifica se stock_code contém, ao menos, uma letra e conta quantos são
df1[df1['stock_code'].str.contains('[a-zA-Z]')]

In [ ]:
# verifica se stock_code contém somente letras e mostra a quantidade de ocorrências
df1.loc[~df1['stock_code'].str.contains('[0-9]'), 'stock_code'].value_counts()

In [ ]:
df1.loc[~df1['stock_code'].str.contains('[0-9]'), 'stock_code'].unique()

#### Country

In [ ]:
df1['country'].value_counts(normalize=True)

<font color='red'>**OBSERVAÇÕES:**</font>
- invoice_no possui:
    - 3839 entradas únicas que possuem letras
    - 9291 entradas totais que possuem letras
    - todas as entradas com letras correspondem à uma quantity negativa, indicando uma operação de cancelamento de venda
- stock_code que possuem somente letras:
    - BANK CHARGES -> qtd: 37
    - AMAZONFEE ->    qtd: 34
    - CRUK ->         qtd: -16
    - D ->            qtd: 77
    - DOT ->          qtd: 710
    - M ->            qtd: 571
    - S ->            qtd: 63
    - POST ->         qtd: 1256
    - DCGSSGIRL ->    qtd: 13
    - DCGSSBOY ->     qtd: 11
    - PADS ->         qtd: 4
    - B ->            qtd: 3
    - m ->            qtd: 1
- description:
    - deletar coluna pois, a princípio, não tem relevância como feature
- country:
    - 91,43% está no Reino Unido

In [ ]:
#profile = ProfileReport(df=df1, title='Profiling report')

In [ ]:
#profile

# 2. Variables Filtering

In [ ]:
df2 = df1.copy()

In [ ]:
df2.query('quantity < 0').sort_values('quantity').head()

In [ ]:
df2.query('customer_id == 20914')

In [ ]:
# selecionar somente preços maiores que 4 centavo
df2 = df2.query('unit_price >= 0.04')

In [ ]:
# retirar alguns stock_codes
codes = ['POST', 'D', 'DOT', 'M', 'BANK CHARGES', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK']
df2 = df2.query('stock_code != @codes')

In [ ]:
df2 = df2.drop(columns='description')

In [ ]:
drop_country = ['European Community', 'Unspecified']
df2 = df2.query('country != @drop_country')

In [ ]:
# separar as quantidades negativas como devoluções (cancelamentos)
df2_returns = df2.query('quantity < 0')
df2_purchases = df2.query('quantity >= 0')

In [ ]:
save_interim(dataframe=df2, file_name='df2.csv')
save_interim(dataframe=df2_returns, file_name='df2_returns.csv')
save_interim(dataframe=df2_purchases, file_name='df2_purchases.csv')

# 3. Feature Engineering

In [ ]:
df3 = df2.copy()

In [ ]:
# data reference - group by customer
df_ref = df3[['customer_id']].drop_duplicates(ignore_index=True)
df_ref.head()

#### **Gross revenue (Faturamento)**

Faturamento para cada produto = quantidade x preço

In [ ]:
df2_purchases['gross_revenue'] = df2_purchases['quantity'] * df2_purchases['unit_price']

In [ ]:
# total value of purchases per customer
df_monetary = df2_purchases[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

#### **Recency**

How recent is the last purchase made by each customer.

In [ ]:
# find the last purchase date of each customer and subtracts of the overall last purchase date
df_last_purchase = df2_purchases[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_last_purchase['recency'] = (df_last_purchase['invoice_date'].max() - df_last_purchase['invoice_date']).dt.days
df_ref = pd.merge(df_ref, df_last_purchase, on='customer_id', how='left')

In [ ]:
df_ref.isna().sum()

#### **Quantity of purchases**

Consider the unique invoice_no (transaction)

In [ ]:
df_freq = df2_purchases[['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index()
df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')
df_ref.head()

In [ ]:
df_ref.isna().sum()

#### **Quantity of products purchased**

In [ ]:
aux = df2_purchases.loc[:, ['customer_id', 'quantity']].drop_duplicates().groupby('customer_id').count().reset_index()
df_ref = pd.merge(df_ref, aux, on='customer_id', how='left')
df_ref.isna().sum()

#### **Average ticket**

Average spend of each client

In [ ]:
df_avg_ticket = df2_purchases[['customer_id', 'gross_revenue']].groupby('customer_id').mean().reset_index().rename(columns={'gross_revenue': 'avg_ticket'})
df_ref = pd.merge(df_ref, df_avg_ticket, how='left', on='customer_id')

In [ ]:
df_ref.isna().sum()

#### **Average recency days**

Average days between purchases

In [ ]:
df_aux = df3[['customer_id', 'invoice_date']].drop_duplicates().sort_values(['customer_id', 'invoice_date'], ascending=[False, False])
df_aux['next_customer_id'] = df_aux['customer_id'].shift()
df_aux['previous_date'] = df_aux['invoice_date'].shift()

df_aux['avg_recency_days'] = df_aux.apply(lambda x: (x['invoice_date'] - x['previous_date']).days if x['customer_id'] == x['next_customer_id'] else np.nan, axis=1)

df_aux = df_aux.drop(['invoice_date', 'next_customer_id', 'previous_date'], axis=1).dropna()

df_avg_recency_days = df_aux.groupby('customer_id').mean().reset_index()

df_ref = pd.merge(df_ref, df_avg_recency_days, on='customer_id', how='left')
df_ref.isna().sum()

#### **Frequency purchase**

In [ ]:
df_aux = df3[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates().groupby('customer_id').agg(
                                                            max_inv_date = ('invoice_date', max),
                                                            min_inv_date = ('invoice_date', min),
                                                            days = ('invoice_date', lambda x: (x.max() - x.min()).days + 1),
                                                            buy = ('invoice_no', 'count')).reset_index()

In [ ]:
# Frequency
df_aux['frequency'] = df_aux[['buy', 'days']].apply(lambda x: x['buy']/x['days'] if x['days'] != 0 else 0, axis=1)

In [ ]:
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left')

df_ref.isna().sum()

#### **Number of returns**

In [ ]:
df_returns = df2_returns[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'returns'})
df_returns['returns'] = df_returns['returns'] * -1

In [ ]:
df_ref= pd.merge(df_ref, df_returns, on='customer_id', how='left')
df_ref.loc[df_ref['returns'].isna(), 'returns'] = 0

In [ ]:
df_ref.isna().sum()

#### **Basket size** (quantidade de itens por cesta)

In [ ]:
df_aux = df2_purchases.loc[:, ['customer_id', 'invoice_no', 'quantity']].groupby('customer_id').agg(
                                                                        n_purchase = ('invoice_no', 'nunique'),
                                                                        n_products = ('quantity', 'sum')).reset_index()

In [ ]:
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

In [ ]:
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'avg_basket_size']], on='customer_id', how='left')
df_ref.isna().sum()

# 4. EDA (Exploratory Data Analysis)

In [ ]:
df2.hist(bins=50, figsize=(10,6))

In [ ]:
df_ref.hist(bins=75, figsize=(10,6))

In [ ]:
df4 = df_ref.dropna().copy()
df4.isna().sum()

# 5. Data Preparation

In [ ]:
df5 = df4.copy()

In [ ]:
std_scaler =StandardScaler()

df5['gross_revenue'] = std_scaler.fit_transform(df5[['gross_revenue']])
df5['recency'] = std_scaler.fit_transform(df5[['recency']])
df5['invoice_no'] = std_scaler.fit_transform(df5[['invoice_no']])
df5['avg_ticket'] = std_scaler.fit_transform(df5[['avg_ticket']])
#df5['returns'] = std_scaler.fit_transform(df5[['returns']])

# 6. Feature Selection

In [ ]:
df6 = df5.copy()

# 7. Hyperparameter Fine Tunning

In [ ]:
df7 = df5.drop(columns=['returns'])
df7.head()

In [ ]:
df7.describe()

## 7.1. Within-Cluster Sum of Squares (WSS)

Returns the compactness of the clusters. It is global measure and does not tell if the clusters are well separated or if there is overlaping between them.

In [ ]:
clusters = [2, 3, 4, 5, 6, 7]
model = KMeans(n_init=10)
elbow = KElbowVisualizer(estimator=model, k=clusters)
elbow.fit(df7)
elbow.show()

In [ ]:
S = KElbowVisualizer(estimator=model, k=clusters, metric='silhouette', timings=False)
S.fit(df7)
S.show()

## 7.2. Silhouette Analysis

Measures how well a point fits into its cluster compared to the others.

In [ ]:
fig, ax = plt.subplots(3,2, figsize=(16,20))

for k in clusters:
    km_model = KMeans(n_clusters=k, init='random', n_init=10, max_iter=100, random_state=42)
    x, y = divmod(k, 2)
    SS = SilhouetteVisualizer(estimator=km_model, colors='yellowbrick', ax=ax[x-1][y])
    SS.fit(df7)
    SS.finalize()
    #SS.show()

# 8. Model Training

## 8.1. K-Means

In [ ]:
k = 3

kmeans = KMeans(n_clusters=k, init='random', n_init=10, max_iter=300, random_state=42)
kmeans.fit(df7)
labels = kmeans.predict(df7)

### 8.1.1. Validation

In [ ]:
# WSS
print(f'WSS: {kmeans.inertia_}')

# SS
print(f'Silhouette score: {silhouette_score(X=df7, labels=labels, metric="euclidean")}')

# 9. Cluster Analisys

In [ ]:
df9 = df7.copy()
df9['cluster'] = labels
df9.head()

## 9.1. Visualization

In [ ]:
df7.head()

In [ ]:
SS_fig = SilhouetteVisualizer(kmeans, colors='sns_dark')
SS_fig.fit(df7)
SS_fig.show()

In [ ]:
#fig = px.scatter_3d(data_frame=df9, x='gross_revenue', y='recency', z='invoice_no', color='cluster', width=600, height=600)
#fig.show()

In [ ]:
df9.head()

### 9.1.1. 2d plot

In [ ]:
df_viz = df9.drop(columns='customer_id')
sns.pairplot(data=df_viz, hue='cluster', corner=True, palette='Set1')

### 9.1.2. UMAP

Ferramenta para visualização de dados com alta dimensionalidade em um plano 2D.
- Clusters com boa definição (bom agrupamento) em 2D indicam um bom agrupamento na alta dimensionalidade

In [ ]:
df9.head()

In [ ]:
reducer = umap.UMAP(n_neighbors=90, n_jobs=-1)
embedding = reducer.fit_transform(df9)

df_viz['embedding_x'] = embedding[:, 0]
df_viz['embedding_y'] = embedding[:, 1]

In [ ]:
sns.scatterplot(x='embedding_x',
                y='embedding_y',
                hue='cluster',
                palette=sns.color_palette('hls', n_colors=len(df_viz['cluster'].unique())),
                data=df_viz)

## 9.2. Cluster profile

In [ ]:
df9['gross_revenue'].sum()

In [ ]:
df9[['cluster', 'gross_revenue']].groupby('cluster').sum() / df9['gross_revenue'].sum()

In [ ]:
# Number of customers
df_clusters = df9[['customer_id', 'cluster']].groupby('cluster').count().reset_index()

# Number of customers (%)
df_clusters['% customer_id'] = (df_clusters[['customer_id']] / df_clusters[['customer_id']].sum()) * 100

# Average gross revenue
aux1 = df9[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()

# % Gross revenue
aux2 = (df9[['cluster', 'gross_revenue']].groupby('cluster').sum() / df9['gross_revenue'].sum()).reset_index().rename(columns={'gross_revenue': '% gross_revenue'})

# Average recency
aux3 = df9[['recency', 'cluster']].groupby('cluster').mean().reset_index()

# Average invoice no
aux4 = df9[['invoice_no', 'cluster']].groupby('cluster').mean().reset_index()

# Average ticket
aux5 = df9[['avg_ticket', 'cluster']].groupby('cluster').mean().reset_index()

In [ ]:
df_clusters = pd.merge(df_clusters, aux1, on='cluster', how='left')
df_clusters = pd.merge(df_clusters, aux2, on='cluster', how='left')
df_clusters = pd.merge(df_clusters, aux3, on='cluster', how='left')
df_clusters = pd.merge(df_clusters, aux4, on='cluster', how='left')
df_clusters = pd.merge(df_clusters, aux5, on='cluster', how='left')
df_clusters

---

**Cluster 1 (candidatos à insiders)**:
- Número de customers: 267 (6,1%)
- Recência média: 20,28
- Média de compras: 19,5
- Média de receita: US$ 8.089,00
- Ticket médio: US$ 62,78
---

**Cluster 2**:
- Número de customers: 6 (0,14%)
- Recência média: 7,16
- Média de compras: 89
- Média de receita: US$ 182.181,98
- Ticket médio: US$ 253,62
---

**Cluster 3**:
- Número de customers: 28 (0,64%)
- Recência média: 6,18
- Média de compras: 57,82
- Média de receita: US$ 42.614,39
- Ticket médio: US$ 162,86
---

**Cluster 4**:
- Número de customers: 4071 (93,1%)
- Recência média: 96,96
- Média de compras: 3,64
- Média de receita: US$ 946,69
- Ticket médio: US$ 25,35
---

# 10. Deploy to Production